# Plotting statistical formula to make sense of them

## Discrete Distributions: The Binomial Distribution

- The variable is for a sample that consists of a fixed number of experimental trials (the sample size).
- The variable has only two mutually exclusive and collectively exhaustive events, typically labeled as success and failure.
- The probability of an event being classified as a success, p, and the probability of an event being classified as a failure, 1 – p, are both constant in all experimental trials.
- The event (success or failure) of any single experimental trial is independent of (not influenced by) the event of any other trial.

In [2]:
import scipy as stats
import statsmodels.api as sm


Descriptive statistics aims to describe and summarise a data set in a meaningful way. Descriptive statistics doesn't make any assumptions about the population as a whole.
Measures of central tendency:
- Mean
- Median
- Mode
Measures of dispersion:

